In [ ]:
!pip install aspect-based-sentiment-analysis
!pip uninstall tensorflow -y
!pip install tensorflow==2.2.0
!pip install deepsegment
!pip install keras==2.3.1
from tensorflow import keras

In [ ]:
#@title Search Videos Using Youtube API

import argparse
from pprint import pprint 

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

DEVELOPER_KEY = '######' # yt developer key
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

def youtube_search(options):
  youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

  # Call the search.list method to retrieve results matching the specified
  # query term.
  search_response = youtube.search().list(
    q=options['q'],
    part=options['part'],
    maxResults=options['maxResults'],
    relevanceLanguage=options['relevanceLanguage'],
    regionCode=options['regionCode']
  ).execute()

  videos = []

  
  # Add each result to the appropriate list, and then display the lists of matching videos
  for search_result in search_response.get('items', []):
    if search_result['id']['kind'] == 'youtube#video':
      videos.append({'title':search_result['snippet']['title'],'id': search_result['id']['videoId']})
  return videos

def get_input():
  input_string = input('Enter product: ')
  input_string += 'review' 
  import re
  asin = input('Enter aspects(comma seperated list): ')
  words = asin.split(',')
  words = words + ['price']
  words = [word.lower().strip() for word in words]
  words = [word.strip() for word in words if word]
  import string
  table = str.maketrans('', '', string.punctuation)
  aspects = list(set([w.translate(table) for w in words]))
  options = {
      'q': input_string,
      'part': 'id,snippet',
      'maxResults': 10,
      'relevanceLanguage': 'en',
      'regionCode': 'US'
  }
  videos = youtube_search(options)
  print('Found following videos: ')
  for video in videos:
    print("ID: "+ video['id']+ "  TITLE: "+video['title'])
  return videos, aspects

In [ ]:
#@title Get Subtitles auto-generated by youtube using api

!pip3 install youtube_transcript_api -q
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter
formatter = TextFormatter()

# videos = [{'id': 'X1b3C2081-Q', 'title': 'iPhone 12 Review: Just Got Real!'}]
def get_subs(videos, max):
  count = 0;
  subtitle_list = []
  title_list = []
  for video in videos:
    # print(video)
    try:
      subs =  YouTubeTranscriptApi.get_transcript(video['id'], languages=['en-GB', 'en','en-US'])
      subtitle_list.append(formatter.format_transcript(subs))
      count = count + 1
      title_list.append(video['title'])
      if count >= max:
        break
    except:  
      print("Could not find subtitles for Video"+ video['title'])

  return subtitle_list, title_list

In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)
from deepsegment import DeepSegment
import aspect_based_sentiment_analysis as absa
from pprint import pprint
import json

def analysis(subtitle, aspects):
  segmenter = DeepSegment('en')

  sentences = segmenter.segment_long(subtitle, n_window=10)
  smolsent = []
  sent =''
  for n, i in enumerate(sentences):
    l = i.split(' ')
    if(len(l) > 250):
      smolsent.append(' '.join(l[:250]))
    else:
      if n != len(sentences)-1:
        if(len(sent.split(' ')) + len(l) < 250):
          sent = sent + i
        else:
          smolsent.append(sent)
          sent = i
      else:
        if(len(sent.split(' ')) + len(l) < 250):
          smolsent.append(sent+i)
        else:
          smolsent.append(sent)
          smolsent.append(i)


  recognizer = absa.aux_models.BasicPatternRecognizer()
  nlp = absa.load(pattern_recognizer=recognizer)
  op={}
  for i in aspects:
      op[i] = {'sentiment':[],'scores':[]}
  for sent in smolsent:
      sentiments = nlp(text=sent, aspects=aspects)
      for i in sentiments:
        if i.aspect in i.text: 
            op[i.aspect]['sentiment'].append(i.sentiment)
            op[i.aspect]['scores'].append(i.scores)

  return op

In [ ]:
import numpy as np
def get_absa(subtitle_list, aspects):
  results = {}
  for i, subtitle in enumerate(subtitle_list):
    op = analysis(subtitle, aspects)
    print(op)
    results[i] = {}
    for aspect in aspects:
      if(op[aspect]['scores']):
        score = np.mean(op[aspect]['scores'], axis=0)
        results[i][aspect] = score
        print(aspect, score)
      else:
        print("Video doesnt mention about", aspect)
  return results

In [ ]:
# pprint(results)

In [ ]:
# import matplotlib.pyplot as plt

# absent_rows = []
# for i in range(len(results)):
#   if not results[i]:
#     absent_rows.append(i)

# print()
# for result,title in zip(results,title_list):
#   if result in absent_rows:
#     continue
#   print()
#   fig, axs = plt.subplots(1,len(aspects),figsize=(10,10))
#   fig.suptitle(title, fontsize=20, fontweight='bold', y=0.8)
#   for i, aspect in enumerate(aspects):
#     labels = ['Neutal','Positive','Negative']
#     if (results[result]):
#       values = results[result][aspect]
#       axs[i].pie(values, labels = labels ,autopct = '%.2f%%')
#       axs[i].set_title(aspect)
#   plt.show()


In [ ]:
import matplotlib.pyplot as plt

def get_plots(results, title_list):
  plots = []
  for result,title in zip(results,title_list):
    if not results[result]:
      continue
    fig, axs = plt.subplots(1,len(aspects),figsize=(15,15))
    fig.suptitle(title, fontsize=20, fontweight='bold', y=0.7)
    if(len(aspects) > 1):
      for i, aspect in enumerate(aspects):
      # for i, aspect in enumerate(results[result].keys()):
        labels = ['Neutral','Positive','Negative']
        if(aspect in results[result]) :
          values = results[result][aspect]
          axs[i].pie(values, labels = labels ,autopct = '%.2f%%')
        else :
          axs[i].pie([], labels = [] ,autopct = '%.2f%%')
        axs[i].set_title(aspect)
      # plt.show()
      plots.append(plt)
    else:
      for i, aspect in enumerate(aspects):
      # for i, aspect in enumerate(results[result].keys()):
        labels = ['Neutal','Positive','Negative']
        if(aspect in results[result]) :
          values = results[result][aspect]
          axs.pie(values, labels = labels ,autopct = '%.2f%%')
        else :
          axs.pie([], labels = [] ,autopct = '%.2f%%')
        axs.set_title(aspect)
      # plt.show()
      plots.append(plt)

  return plots

In [ ]:
from IPython.core.display import HTML
import math
def get_html(results,title_list):
  html = ''
  for result,title in zip(results,title_list):
    html = html + '<br><b>'+ title + '</b><br>'
    if not results[result]:
      continue
    for i, aspect in enumerate(results[result].keys()):
      html += '<span style="text-transform:capitalize">' + aspect + '</span>' + ': ' + '★'* math.ceil((results[result][aspect][1] + results[result][aspect][0])*5)
      html += '☆'*( 5 - math.ceil((results[result][aspect][1] + results[result][aspect][0])*5) )
      html = html + '<br>'

  return(html)

In [ ]:
videos, aspects = get_input()

Enter product: iphone 12
Enter aspects(comma seperated list): battery
Found following videos: 
ID: X1b3C2081-Q  TITLE: iPhone 12 Review: Just Got Real!
ID: uUIFK7i9lek  TITLE: iPhone 12 Review - 6 Months Later
ID: OCnzH7IiHo0  TITLE: iPhone 12 review: new standard
ID: N8Ala1A31GY  TITLE: iPhone 12 Review - 1 Month Later
ID: PrfZ1j6Ls3I  TITLE: Apple iPhone 12 Review | One Month Later
ID: g0AOf2AZ-8w  TITLE: iPhone 12 - The iPhone is New Again
ID: tpaqH4KJNxk  TITLE: iPhone 12 Review After 1 Week - The Facts
ID: TJqFee9Kit4  TITLE: iPhone 12 Long-Term Review: Buy or Wait for iPhone 13..?
ID: 7h0CXQgPqN0  TITLE: Apple iPhone 12 full review
ID: Wb5OXtrPGLE  TITLE: Apple iPhone 12 Review - Yes, Buy it!


In [ ]:
max_subs = 1
subtitle_list, title_list = get_subs(videos, max_subs)
results = get_absa(subtitle_list, aspects)

In [ ]:
plots = get_plots(results, title_list)
for plot in plots:
  plot.show()

In [ ]:
html = get_html(results,title_list)
HTML(html)